In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'Depot/Code/song_play'))
	print(os.getcwd())
except:
	pass


 # ETL Processes
 Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from pathlib import Path
from sql_queries import *



In [2]:
conn = psycopg2.connect(
    "host=127.0.0.1 dbname=sparkifydb user=student password=student"
)
conn.set_session(autocommit=True)
cur = conn.cursor()



In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, "*.json"))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files



 # Process `song_data`
 In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

 Let's perform ETL on a single song file and load a single record into each table to start.
 - Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
 - Select the first song in this list
 - Read the song file and view the data

In [4]:
song_root_dir = Path().cwd() / "data" / "song_data"
song_files = get_files(song_root_dir)
len(song_files)



71

In [5]:
filepath = song_files[0]
print(filepath)



/Users/rob/Depot/Code/song_play/data/song_data/A/A/A/TRAAAEF128F4273421.json


In [6]:
df = pd.read_json(filepath, lines=True)
df.columns


Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

 ## #1: `songs` Table
 #### Extract Data for Songs Table
 - Select columns for song ID, title, artist ID, year, and duration
 - Use `df.values` to select just the values from the dataframe
 - Index to select the first (only) record in the dataframe
 - Convert the array to a list and set it to `song_data`

In [7]:
# alternate method: select columns and return as a tuple knowing that there is one song per dataframe
# note: results in year as typye np.int64 and duration as type np.float64
song_data = next(
    df[["song_id", "title", "artist_id", "year", "duration"]].itertuples(
        index=False, name=None
    )
)
song_data


('SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363)

In [8]:
# recommended method: select columns, select first row, get values as numpy array and convert to a list
# note: results in year as typye int and duration as type float
song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values[0].tolist()
song_data

# alternative that results in np.int64 and np.float64 numeric types apparently due to using iloc
# see: https://knowledge.udacity.com/questions/38150
# song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].iloc[0, :].values.tolist()

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [9]:
# drop and re-create all tables
for sql_cmd in drop_table_queries + create_table_queries:
    cur.execute(sql_cmd)


 #### Insert Record into Song Table
 Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
# how should I handle missing values?  Here is one way:
# replace missing values with None since pd.read_json does not handle missing value conversion
song_data = [x if x else None for x in song_data]
[type(x) if x else None for x in song_data]


[str, str, str, int, float]

In [11]:
cur.execute(song_table_insert, song_data)



 Run `test.ipynb` to see if you've successfully added a record to this table.

 ## #2: `artists` Table
 #### Extract Data for Artists Table
 - Select columns for artist ID, name, location, latitude, and longitude
 - Use `df.values` to select just the values from the dataframe
 - Index to select the first (only) record in the dataframe
 - Convert the array to a list and set it to `artist_data`

In [12]:
print(df.columns)
artist_data = (
    df[
        [
            "artist_id",
            "artist_name",
            "artist_location",
            "artist_latitude",
            "artist_longitude",
        ]
    ]
    .values[0]
    .tolist()
)
artist_data


Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')


['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]

 #### Insert Record into Artist Table
 Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()


 Run `test.ipynb` to see if you've successfully added a record to this table.

 # Process `log_data`
 In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

 Let's perform ETL on a single log file and load a single record into each table.
 - Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
 - Select the first log file in this list
 - Read the log file and view the data

In [14]:
log_data_root = Path().cwd() / "data" / "log_data"
log_files = get_files(log_data_root)
len(log_files)



30

In [15]:
filepath = log_files[0]


In [16]:
df = pd.read_json(filepath, lines=True)
before = df.shape
df = df.loc[df.page.isin(["NextSong"])]
after = df.shape
f"before: {before}  after:{after}"


'before: (95, 18)  after:(73, 18)'

 ## #3: `time` Table
 #### Extract Data for Time Table
 - Filter records by `NextSong` action
 - Convert the `ts` timestamp column to datetime
   - Hint: the current timestamp is in milliseconds
 - Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_df` to a list containing these values in order
   - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
 - Specify labels for these columns and set to `column_labels`
 - Create a dataframe, `time_data,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:

# cite: https://pandas.pydata.org/pandas-docs/stable/reference/series.html#time-series-related
df = df.assign(timestamp=pd.to_datetime(df.ts, unit="ms"))
df.timestamp = df.timestamp.dt.tz_localize("UTC")
df.head(10)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:33:56.796000+00:00
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:36:10.796000+00:00
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:40:34.796000+00:00
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11 04:36:13.796000+00:00
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75,2018-11-11 04:36:46.796000+00:00
6,Gary Allan,Logged In,Ryann,F,0,Smith,259.83955,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,The One,200,1541930188796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 09:56:28.796000+00:00
7,Miracle Fortress,Logged In,Ryann,F,1,Smith,200.90730,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Five Roses,200,1541930447796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:00:47.796000+00:00
8,Don Omar,Logged In,Ryann,F,2,Smith,261.35465,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Cuentale,200,1541930647796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:04:07.796000+00:00
9,Jay-Z,Logged In,Ryann,F,3,Smith,212.27057,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,D'Evils,200,1541930908796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:08:28.796000+00:00
10,Red Hot Chili Peppers,Logged In,Ryann,F,4,Smith,231.33995,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Easily (Album Version),200,1541931120796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:12:00.796000+00:00


In [18]:
time_df = pd.DataFrame(
    {
        "timestamp": df.timestamp,
        "hour": df.timestamp.dt.hour,
        "day": df.timestamp.dt.day,
        "week_of_year": df.timestamp.dt.week,
        "month": df.timestamp.dt.month,
        "year": df.timestamp.dt.year,
        "weekday": df.timestamp.dt.weekday,
    }
)
time_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 94
Data columns (total 7 columns):
timestamp       73 non-null datetime64[ns, UTC]
hour            73 non-null int64
day             73 non-null int64
week_of_year    73 non-null int64
month           73 non-null int64
year            73 non-null int64
weekday         73 non-null int64
dtypes: datetime64[ns, UTC](1), int64(6)
memory usage: 4.6 KB


 #### Insert Records into Time Table
 Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))


 Run `test.ipynb` to see if you've successfully added records to this table.

 ## #4: `users` Table
 #### Extract Data for Users Table
 - Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
df.columns


Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId',
       'timestamp'],
      dtype='object')

In [21]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df = user_df.drop_duplicates(subset="userId", keep="last")
user_df.shape


(15, 5)

 #### Insert Records into Users Table
 Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()


 Run `test.ipynb` to see if you've successfully added records to this table.

 ## #5: `songplays` Table
 #### Extract Data and Songplays Table
 This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

 - Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.

 - Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

 #### Insert Records into Songplays Table
 - Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print(f"row={row}\n\n")

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

        print(
            f"""
            row.userId={row.userId}
            songid={songid}
            artistid={artistid}
            row.sessionId={row.sessionId}
            row.ts={row.ts/1000}
            row.level={row.level}
            row.location={row.location}
            row.userAgent={row.userAgent}
            """
        )

    # insert songplay record
    songplay_data = (
        row.userId,
        songid,
        artistid,
        row.sessionId,
        row.ts / 1000,
        row.level,
        row.location,
        row.userAgent,
    )
    print(song_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()


row=artist                                                    Frumpies
auth                                                     Logged In
firstName                                                 Anabelle
gender                                                           F
itemInSession                                                    0
lastName                                                   Simpson
length                                                     134.478
level                                                         free
location               Philadelphia-Camden-Wilmington, PA-NJ-DE-MD
method                                                         PUT
page                                                      NextSong
registration                                           1.54104e+12
sessionId                                                      455
song                                                    Fuck Kitty
status                                                    

SyntaxError: syntax error at or near "CAST"
LINE 7:     CAST(start_time AS timestamp),
            ^


 Run `test.ipynb` to see if you've successfully added records to this table.

 # Close Connection to Sparkify Database

In [24]:
conn.close()


 # Implement `etl.py`
 Use what you've completed in this notebook to implement `etl.py`.